## **<span style="color:purple">결정 트리</span>**
---

#### **로지스틱 회귀로 와인 분류하기**

#### **설명하기 쉬운 모델과 어려운 모델**
- 로지스틱 회귀로 분류하는 것은 결과를 설명하기 어려운 단점이 있음

#### **결정 트리**
- 루프 노드: 맨 위의 노드
- 리프 노드: 맨 아래 끝에 달린 노드
- 노드: 훈련 데이터의 특성에 대한 테스트를 표현
- 표준화 전처리를 할 필요가 없음

#### **불순도**
- $지니 불순도=(1-(음성 클래스 비율^2 + 양성 클래스 비율^2))$
- 지니 불순도가 0.5일 때 최악이고, 0에 가까울 수록 좋음. 0인 노드를 순수 노드라 함
- 결정 트리 모델은 부모 노드와 자식 노드의 불순도 차이가 가능한 크도록 트리를 성장시킴
- $불순도 차이(정보 이득)=(왼쪽 노드 샘플 수 / 부모의 샘플 수)\times왼쪽 노드 불순도 - (오른쪽 노드 샘플 수 / 부모의 샘플 수)\times오른쪽 노드 불순도$
- 엔트로피 불순도도 있음

#### **가지치기**
- 가지치기를 하지 않으면 과대적합되기 쉬움

## **<span style="color:purple">교차 검증과 그리드 서치</span>**
---
- 테스트 세트를 많이 사용하면, 테스트 세트로 일반화 성능을 예측하는 꼴이 됨

#### **검증 세트**
- 검증 세트를 활용해 가장 좋은 모델을 고르고, 마지막에 테스트 세트에서 최종 점수를 평가함

#### **교차 검증**
- 검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복하고 이 점수들을 평균하여 최종 검증 점수를 얻음
- train_test_split() 함수는 전체 데이터를 섞은 후 나눠주기 때문에, 교차 검증시 훈련 세트를 섞을 필요가 없음
- 그러나 만약 교차 검증시 훈련 세트를 섞으려면 분할기를 지정해야함

#### **하이퍼파라미터 튜닝**
- 매개변수들을 동시에 바꿔가며 최적의 값을 찾아야 함
- 그리드 서치 사용
- GridSearchCV 클래스는 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행함

1. 먼저 탐색할 매개변수를 지정함
2. 그다음 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾음. 이 조합은 그리드 서치 객체에 저장됨
3. 그리드 서치는 최상의 매개변수에서 전체 훈련 세트를 사용해 최종 모델을 훈련함. 이 모델도 그리드 서치 객체에 저장됨

#### **랜덤 서치**
- 매개변수의 값의 범위나 간격을 미리 정하기 어려움
- 랜덤 서치에는 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달함

In [ ]:
from scipy.stats import uniform,randint
rgen=randint(0,10)
rgen.rvs(10)

array([6, 8, 2, 2, 6, 8, 2, 3, 6, 4], dtype=int64)

In [ ]:
np.unique(rgen.rvs(1000),return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64),
 array([104, 112, 101, 100,  99,  93,  99,  97, 109,  86], dtype=int64))

In [ ]:
ugen=uniform(0,1)
ugen.rvs(10)

array([0.11325921, 0.71383778, 0.34450042, 0.23339286, 0.97805413,
       0.95720056, 0.68797648, 0.81293759, 0.35504935, 0.59273014])

In [ ]:
params={"min_impurity_decrease":uniform(0.0001,0.001),
       "max_depth":randint(20,50),
       "min_samples_split":randint(2,25),
       "min_samples_leaf":randint(1,25)}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
gs=RandomizedSearchCV(DecisionTreeClassifier(random_state=42),params,n_iter=100,n_jobs=-1,random_state=42)
gs.fit(train_input,train_target)
print(gs.best_params_)
print(np.max(gs.cv_results_["mean_test_score"]))

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
0.8695428296438884


In [ ]:
dt=gs.best_estimator_
print(dt.score(test_input,test_target))

0.86


## **<span style="color:purple">트리의 앙상블</span>**
---
- 정형 데이터: 어떤 구조로 되어 있다는 뜻. csv, 데이터베이스, 엑셀에 저장하기 쉬움
- 비정형 데이터: 데이터베이스나 엑셀로 표현하기 어려운 것들
- 앙상블 학습: 정형 데이터를 다루는 데 가장 뛰어난 성과를 내는 알고리즘

#### **랜덤 포레스트**
- 앙상블 학습의 대표 주자이며 안정적인 성능을 가짐
- 결정 트리를 랜덤하게 만들어 결정 트리의 숲을 만듬. 그리고 각 결정 트리의 예측을 사용해 최종 예측을 만듬
- 부트스트램 샘플: 중복적으로 샘플을 뽑아서 만든 샘플
- 각 노드를 분할할 때 전체 특성 중에서 일부 특성을 무작위로 고른 다음 최선의 분할을 찾음
- 과대적합되는 것을 막아주고 검증 세트와 테스트 세트에서 안정적인 성능을 얻을 수 있음
- RandomForestClassifier 클래스는 기본적으로 100개의 결정 트리를 사용함

- OOB(부트스트랩 샘플에 포함되지 않은 샘플)샘플로 결정 트리를 평가할 수 있음. 검증 세트의 역할을 함

#### **엑스트라 트리**
- 랜덤 포레스트와 매우 비슷하게 동작
- 랜덤 포레스트와의 차이점은 부트스트랩 샘플을 사용하지 않고 전체 훈련 세트를 사용함
- 노드를 분할할 때 가장 좋은 분할을 찾는 것이 아니라 무작위로 분할함 -> 더 많은 트리가 필요함
- 성능이 낮아지겠지만 많은 트리를 앙상블 하기 때문에 과대적합을 막고 검증 세트의 점수를 높이는 효과가 있음

#### **그레디언트 부스팅**
- 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법
- GradientBoostingClassifier은 기본적으로 깊이가 3인 결정 트리를 100개 사용함
- 경사 하강법을 사용하여 트리를 앙상브레 추가함
- 분류에서는 로지스틱 손실 함수를 사용하고 회귀에서는 평균 제곱 오차 함수를 사용함
- 트리의 개수를 늘려도 과대적합에 매우 강함
- 학습률의 기본값은 0.1임. 클 수록 복잡하고 훈련 세트에 과대적합된 모델을 얻을 수 있음

#### **히스토그램 기반 그레이디언트 부스팅**
- 정형 데이터를 다루는 머신러닝 알고리즘 중에 가장 인기가 높은 알고리즘
- 입력 특성을 256개의 구간으로 나눔 -> 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있음
- 입력에 누락된 특성이 있더라도 이를 따로 전처리할 필요가 없음
- n_estimators 대신에 max_iter 매개변수를 사용함
- 과대적합을 잘 억제하면서 그레이디언트 부스팅보다 조금 더 높은 성능을 제공함

- 그레이디언트 부스팅 알고리즘을 구현한 대표적인 라이브러리는 XGBoost임